In [203]:
import numpy as np

### LISA

In [204]:
from PIL import Image
import os
import pandas as pd

Open root directory:

In [205]:
root = '/Users/brian/Documents/datasets/lisa_set'

Create a list of the-sub directories that have images:

In [206]:
image_dirs = [str(x) for x in os.listdir(root) if x[-1].isdigit()]

Get all further sub-directories with images in them:

In [259]:
image_paths = []
for direct in image_dirs:
    for subdir in os.listdir(root+'/'+direct)[:]:
        if subdir[0].isalpha():
            image_paths.append(root+'/'+direct+'/'+subdir+'/')

In [209]:
bounding = 'frameAnnotations.csv' # String name for the annotations

In [274]:
for i in range(len(image_paths)):
    make_images(i)
    print "Finished set: {}".format(i)

Starting set: 0
Creating path: /Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_0
Finished set: 0
Starting set: 1
Creating path: /Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_1
Finished set: 1
Starting set: 2
Creating path: /Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_2
Finished set: 2
Starting set: 3
Creating path: /Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_3
Finished set: 3
Starting set: 4
Creating path: /Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_4
Finished set: 4
Starting set: 5
Creating path: /Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_5
Finished set: 5
Starting set: 6
Creating path: /Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_6
Finished set: 6
Starting set: 7
Creating path: /Users/bri

In [273]:
def make_images(current_ind):
    print "Starting set: " + str(current_ind)
    current = image_paths[current_ind]
    images = [x for x in os.listdir(current) if '.png' in x]
    image_annot = pd.read_csv(current+bounding, delimiter=';')

    save_path = r'/Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_{}'.format(current_ind) 
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        print "Creating path: " + save_path
    with open('data/fullset/set_{}/path.txt'.format(current_ind), 'w') as f:
        f.write(image_paths[i])

    image_bounds_1 = {}
    for row in range(len(image_annot)):
        image_bounds_1[image_annot.loc[row][0]] = (image_annot.loc[row][1],[float(x) for x in str(image_annot.loc[row][2:6]).split() if x.isdigit()])

    saved_images = []
    for im in image_bounds_1.keys():
        img = Image.open(current+im)
        img2 = img.crop(tuple(image_bounds_1[im][1]))
        longer_side = max(img2.size)
        horizontal_padding = (longer_side - img2.size[0]) / 2
        vertical_padding = (longer_side - img2.size[1]) / 2
        img3 = img2.crop(
        (
            -horizontal_padding,
            -vertical_padding,
            img2.size[0] + horizontal_padding,
            img2.size[1] + vertical_padding
        )
        )
        if img3.size[0] > img3.size[1]:
            img4 = img3.crop((0,0,img3.size[1],img3.size[1]))
        elif img3.size[0] < img3.size[1]:
            img4 = img3.crop((0,0,img3.size[0],img3.size[0]))
        else:
            img4 = img3
        image_str = '{}{}'.format(image_bounds_1[im][0], str(img4.size))
        x = 0
        while image_str + str(x) in saved_images:
            x += 1
        image_str += str(x)
        saved_images.append(image_str)
        img4.save("./data/fullset/set_{}/{}.jpg".format(current_ind, image_str))
    

### Mapillary

In [1]:
import requests
from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/')

In [3]:
db = client.mapillary

In [4]:
signs = db.signs

In [52]:
signs_list = []
for i in range(100):
    signs_list.append(signs.find_one()['features'][i]['properties']['value'])
signs_list = np.array(signs_list)

In [56]:
for i in np.unique(signs_list):
    print i

error_import-warning--road-narrows-right--us
regulatory--dual-lanes-bicyclists-and-pedestrians--g1
regulatory--no-entry--g1
regulatory--no-equestrians--g1
regulatory--no-left-turn--g1
regulatory--no-parking--g2
regulatory--no-right-turn--g1
regulatory--no-u-turn--g1
regulatory--stop--g1
regulatory--yield--g1
warning--added-lane-right--g1
warning--curve-left--g2
warning--curve-right--g2
warning--double-curve-first-left--g2
warning--double-curve-first-right--g2
warning--double-turn-first-right--g1
warning--height-restriction--g2
warning--junction-with-a-side-road-acute-left--g1
warning--loop-270-degree--g1
warning--pedestrians-crossing--g4
warning--road-narrows-left--g2
warning--road-narrows-right--g2
warning--slippery-road-surface--g2
warning--texts--g2
warning--texts--g3
warning--traffic-merges-right--g1
warning--traffic-signals--g3
warning--turn-left--g1
warning--turn-right--g1


### Flickr:

In [79]:
import flickrapi
import re
import json
import pprint

In [34]:
with open('/Users/brian/Documents/keys/flickr.txt') as f:
    keys = f.read()
keys = keys.split()
key = re.search(r"'([^']*?)'",keys[0]).group().strip("'")
secret = re.search(r"'([^']*?)'",keys[1]).group().strip("'")

In [102]:
flickr = flickrapi.FlickrAPI(key,secret)
photos = flickr.photos.search(tags='sybren,365,threesixtyfive', per_page='10', format='json')

In [76]:
sets = flickr.photosets.getList(user_id='73509078@N00', format='json')

In [103]:
set_json = json.loads(sets)
photos_j = json.loads(photos)

In [104]:
set_json['photosets']['photoset'][0]

{u'can_comment': 0,
 u'count_comments': u'0',
 u'count_views': u'36',
 u'date_create': u'1475529863',
 u'date_update': u'1475530523',
 u'description': {u'_content': u'Photos I take en route to work.'},
 u'farm': 9,
 u'id': u'72157674639812906',
 u'needs_interstitial': 0,
 u'photos': u'7',
 u'primary': u'30014551651',
 u'secret': u'cfdd173178',
 u'server': u'8742',
 u'title': {u'_content': u'En Route'},
 u'videos': 0,
 u'visibility_can_see_set': 1}

In [105]:
photos_j['photos']['photo'][0]

{u'farm': 4,
 u'id': u'33404189525',
 u'isfamily': 0,
 u'isfriend': 0,
 u'ispublic': 1,
 u'owner': u'144876000@N07',
 u'secret': u'551dfa8c9e',
 u'server': u'3767',
 u'title': u'70/365'}

https://farm{farm-id}.staticflickr.com/{server-id}/{id}_{secret}.jpg

In [ ]:
https://farm6.staticflickr.com/5631/30014555241_cc74e7f525.jpg